# Optimised Numeric Integration

## Reference Implementation

This is intended to be a clear and easy to read implementation.

In [1]:
import numpy as np
from scipy.integrate import dblquad
from scipy.stats import trapezoid, triang
import time

# Used for tighter inner integral bounds
def x_lower(y):
    # Support of K
    k_range = [np.maximum(y, 0), np.minimum(1, 1+y)]

    # Support of C1
    c1_range = [0, 1]

    # Conventional trapezoid parameters
    a = c1_range[0] - k_range[1]
    
    return a

def x_upper(y):
    # Support of K
    k_range = [np.maximum(y, 0), np.minimum(1, 1+y)]

    # Support of C1
    c1_range = [0, 1]

    # Conventional trapezoid parameters
    d = c1_range[1] - k_range[0]
    
    return np.minimum(d, 0)

def f(y, x):
    # Y marginal
    y_density = triang.pdf(y, loc=-1, scale=2, c=0.5)

    # Support of K
    k_range = [np.maximum(y, 0), np.minimum(1, 1+y)]

    # Support of C1
    c1_range = [0, 1]

    # Conventional trapezoid parameters
    a = c1_range[0] - k_range[1]
    b = c1_range[0] - k_range[0]
    c = c1_range[1] - k_range[1]
    d = c1_range[1] - k_range[0]

    # Scipy trapezoid parameters
    loc = a
    scale = d-a
    c1 = (b-a) / scale
    d1 = (c-a) / scale

    # X conditional probability density
    x_density = trapezoid.pdf(x,c=c1, d=d1,loc=loc, scale=scale)

    return y_density*x_density

s = time.time()
p = dblquad(f, a=-1, b=0, gfun=x_lower, hfun=x_upper)[0]
print(f"Probability: {p*2}")
print(f"Calculated in {time.time()-s:.4f} seconds")

Probability: 0.3333333333333333
Calculated in 0.0693 seconds


## Optimised Python Implementation

Using scipy's stats classes are quite slow, this version constructs PDF for the distributions manually.

In [2]:
from scipy.integrate import dblquad
import time
from math import fabs
s = time.time()

def x_lower(y):
    return 0 - (1+y if y<0 else 1)

def x_upper(y):
    d = 1 - (y if y>0 else 0)
    return 0 if d>0 else d

def f(y, x):
    # Y marginal
    y_density = -fabs(y) + 1

    # Support of K
    k_range0 = y if y>0 else 0
    k_range1 = 1+y if y<0 else 1

    # Conventional trapezoid parameters
    a = -k_range1
    b = -k_range0
    c = 1 - k_range1
    d = 1 - k_range0

    # X conditional probability density
    x_density = (2/(d+c-a-b))
    
    if x < b:
        x_density = x_density*((x-a)/(b-a))
        x_density = x_density if x_density>0 else 0
    elif x > c:
        x_density = x_density*((d-x)/(d-c))
        x_density = x_density if x_density>0 else 0
    
    return y_density*x_density

tol = 1e-06 # Increase for even faster results
st = time.time()
p = dblquad(f, a=-1, b=0, gfun=x_lower, hfun=x_upper)[0]
en = time.time()
print(f"Probability: {p*2}")
print(f"Calculated in {en-st:.4f} seconds")

Probability: 0.3333333333333333
Calculated in 0.0003 seconds


## Optimised Cython Implementation

For fun I tried to squeeze even more performance by writing a compiled implementation.

In [3]:
%load_ext cython

In [4]:
%%cython
from scipy.integrate import dblquad
from libc.math cimport abs
import cython
import time
import numpy as np

def x_lower(y):
    return 0 - (1+y if y<0 else 1)

def x_upper(y):
    d = 1 - (y if y>0 else 0)
    return 0 if d>0 else d

@cython.cdivision(True)
def f(y: cython.double, x: cython.double):
    # Y marginal
    cdef double y_density = -abs(y) + 1

    # Support of K
    cdef double k_range0 = y if y>0 else 0.0
    cdef double k_range1 = 1+y if y<0 else 1.0

    # Conventional trapezoid parameters
    cdef double a = 0.0 - k_range1
    cdef double b = 0.0 - k_range0
    cdef double c = 1.0 - k_range1
    cdef double d = 1.0 - k_range0

    # X conditional probability density
    cdef double x_density = (2.0/(d+c-a-b))
    
    if x < b:
        x_density = x_density*((x-a)/(b-a))
        x_density = x_density if x_density>0 else 0.0
    elif x > c:
        x_density = x_density*((d-x)/(d-c))
        x_density = x_density if x_density>0 else 0.0
    
    return y_density*x_density

tol = 1e-06 # Increase for even faster results
st = time.time()
p = dblquad(f, a=-1, b=0, gfun=x_lower, hfun=x_upper)[0]
en = time.time()
print(f"Probability: {p*2}")
print(f"Calculated in {en-st:.4f} seconds")

Probability: 0.3333333333333333
Calculated in 0.0001 seconds
